In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [4]:
mnist = input_data.read_data_sets("MNIST_data",one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [9]:
#每批次的大小
batch_size = 100
#计算一共有多少批次
n_batch = mnist.train.num_examples // batch_size


#初始化权值函数
def weight_variable(shape):
    #生成截断的正态分布,shape表示生成张量的维度，mean均值，stddev标准差，由自己设定
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

#初始化偏置值函数
def bias_variable(shape):
    #生成一个常量
    initial = tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

#卷积层函数,x输入，W卷积核
def conv2d(x,W):
    #提供神经网络相关操作tf.nn 边界考虑填充0，则输出图片大小与收入图片大小一样
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')

#池化层函数  (n-f+2p/s)+1
def max_pool2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

#定义两个placeholder
#考虑每个图片28x28，行不确定，列784
x = tf.placeholder(tf.float32,[None,784]) #28*28
y = tf.placeholder(tf.float32,[None,10])

#改变x的格式转为4D的向量[batch, in_height, in_width, in_channels]
#就是将x一组图像矩阵重构成新的矩阵
#假设有50张，大小56x56，则batch=50x56x56/28/28/1=200
#则x_image的维数（200，28，28，1）
x_image = tf.reshape(x,[-1,28,28,1])

#初始化第一个卷积层的权值和偏量
#5x5的采样窗口，32个卷积核从一个平面抽取特征
W_conv1 = weight_variable([5,5,1,32])
#每一个卷积核一个偏置值
b_conv1 = bias_variable([32])

#把x_image和权重向量进行卷积(采用卷积函数)，再加上偏置值
#然后应用于relu激励函数
h_conv1 = tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)
#最后进行最大池化，完后第一次卷积
h_pool1 = max_pool2x2(h_conv1)

#初始化第二个卷积层的权重和偏值
#采取5x5的采样窗口，64个卷积核，从32个平面抽取特征
W_conv2 = weight_variable([5,5,32,64])
#每一个卷积核一个偏置值
b_conv2 = bias_variable([64])

#把上一次卷积值h_pool1和权值向量进行第二次卷积操作
#加上偏值，然后应用于relu激励函数
h_conv2 = tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)
h_pool2 = max_pool2x2(h_conv2) #进行最大池化操作

#28x28的图像的第一次卷积后还是28x28，第一次池化后变成14x14 (28-2)/2+1，最后得到32张14x14的平面(有32个卷积核
#1第二次卷积后为14x14，第二次池化后变成7x7 (14-2)/2+1，最后得到64张7x7的平面(有64个卷积核)
#因此，得到64张7x7的平面

#初始化第一个全连接的权值
#上一层有7x7x64个神经元，全连接有1000个神经元
W_fc1 = weight_variable([7*7*64,100])
b_fc1 = bias_variable([100]) 

#把池化层扁平化为1维,7x7x64 图片的像素总量x卷积核
h_pool2_flat = tf.reshape(h_pool2,[-1,7*7*64])
#求第一个全连接层的输出
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1) + b_fc1)

#Dropout层，减少全连接节点，防止出现过拟合，减少模型计算量，增强泛化能力
#keep_prob用来表示神经元的输出概率
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1,keep_prob)

#初始化第二个全连接层
W_fc2 = weight_variable([100,10])
b_fc2 = bias_variable([10])

#计算输出
prediction = tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2) + b_fc2)

#交叉熵代价函数
cross_entropy = tf.reduce_mean(
                tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))

#使用AdamOptimizier进行优化
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

#结果存放在一个布尔列表中
#argmax返回一维张量中最大的值所在的位置
correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))

#求准确率
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

with tf.Session() as sess:
    #设置变量初始化
    sess.run(tf.global_variables_initializer())
    for epoch in range(2):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:batch_xs,y:batch_ys,keep_prob:0.5})
        acc=sess.run(accuracy,feed_dict={x:mnist.test.images, y:mnist.test.labels,keep_prob:1.0})
        print("Iter "+str(epoch) + ", Testing Accuracy= " +str(acc))

Iter 0, Testing Accuracy= 0.901
Iter 1, Testing Accuracy= 0.9424
